# Springboard Intermediate Datascience - Capstone Project
## Housing price prediction based on web data - Scraping

Here below the code used to get the data used for this project. Please do not use this code to avoid generating un-necessary trafic.

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
#To handle exceptions
from selenium.common.exceptions import NoSuchElementException

#-----------------------------------------------------------------------------------------------------
#function that scrapes data from the website url and returns a dataframe
#-----------------------------------------------------------------------------------------------------
def scrape_data(url):#try passing the driver as arg
#     driver = webdriver.Chrome(executable_path="chromedriver") #please refer the path to the chrome driver relatively to this notebooks location
    driver.get(url)
    
    time.sleep(1)
    
    content_element = driver.find_element_by_id("iw-propertypage-verticals")
    content_html = content_element.get_attribute("innerHTML")

    soup = BeautifulSoup(content_html, "html.parser")
    tag = soup.find_all('table')

    content_element2 = driver.find_element_by_id("propertyPage-title-address")
    content_html2 = content_element2.get_attribute("innerHTML")

    soup_address = BeautifulSoup(content_html2,"html.parser")
    address = soup_address.find_all(id='propertyPage-title-address-output')

#     driver.close()

    all_list = []
    all_completed = []

    #listing all elements found in a list of lists, with next to the element the tag it's related to
    for row in soup.find_all(['th','td','tr'],{'class':['iw-propertypage-verticals-characteristic-label','iw-propertypage-verticals-characteristic-value','integrated-service-link']}):
        all_list.append([row.name,(row.text.strip('\n')).strip()])

    #I know all names (col names) are with a th tag. So if two th tags are next to each other I need to introduce an empty string!

    #store first tag as 'td' so that it does not find a mistake on first title which is 'th' tag
    initial = 'td'
    for i in all_list:
        if i[0] == initial:
            all_completed.append(['td',''])
            all_completed.append(i)
            initial='th'
        else:
            all_completed.append(i)
            initial=i[0]

    #Now that I have my list with pairs (th-td or th-tr), I can split it and create a table.
    colnames = [all_completed[i][1] for i in range(0,len(all_completed),2)]
    values = [all_completed[i][1] for i in range(1,len(all_completed),2)]
    house_info = pd.DataFrame([values],columns=colnames)
    #now adding the address
    house_info.loc[:, 'Address'] = [(x.text.strip('\n')).strip().replace('\n','') for x in address] or None
    return house_info


#-----------------------------------------------------------------------------------------------------
#FUNCTION that scrapes all urls from a search page (so all links to houses for sale in my case)
#-----------------------------------------------------------------------------------------------------
def get_all_urls_from_page(page_url):
#     driver = webdriver.Chrome(executable_path="chromedriver") #please refer the path to the chrome driver relatively to this notebooks location
    driver.get(page_url)

    content_element = driver.find_element_by_id('result')
    content_html = content_element.get_attribute("innerHTML")

    soup = BeautifulSoup(content_html, "html.parser")

#     driver.close()
    
    #create list to store all urls, then add all urls to it
    urls=[]
    for div in soup.find_all('div',{'class':'result-xl-bottom-bar'}):
        urls.append(div.a['href'])
    return urls


#store all urls from all pages in list "urls"
#There are 17 pages in the search at time of scraping

#creating list to store alls urls from all search pages
urls = []

In [49]:
#opening driver here to keep it opened through all iterations

chrome_path = r"C:\Users\gmignot\Desktop\chromedriver_win32\chromedriver.exe"

driver = webdriver.Chrome(chrome_path)

In [ ]:
#driver = webdriver.Chrome(executable_path="chromedriver") #please refer the path to the chrome driver relatively to this notebooks location

for page in range(1,18):
    search_page = 'https://www.WEBSITE.be/en/search/house/for-sale?page=' + str(page)
    urls.append(get_all_urls_from_page(search_page))
# driver.close()


In [ ]:
#create our database (empty for now)
database = pd.DataFrame()

#make urls flat
urls = [urls[i][j] for i in range(0,len(urls)) for j in range(0,len(urls[i]))]

urls_done = []

In [29]:
n_unscraped = 0
unscraped_urls = urls[:]

In [ ]:
#open driver to scrape all data from all links in urls - It is already opened above
# driver = webdriver.Chrome(executable_path="chromedriver") #please refer the path to the chrome driver relatively to this notebooks location


for url in unscraped_urls:
    
    if url in urls_done:
        unscraped_urls.remove(url)
        continue
    
    print(f'{len(unscraped_urls)} urls remaining...')
    
    try:
        
        data = scrape_data(url)
        database = pd.concat([database, data], axis=0)
        
        try:
            unscraped_urls.remove(url)
        except:
            urls_done.append(url)
        
    except NoSuchElementException as e: #this exception is thrown sometimes so we skip when it happens
        print(e)
        n_unscraped += 1
        time.sleep(10)
        
        
#close driver only when all houses are scraped        
driver.close()

In [60]:
#saving dataframe to excel file
database.to_excel('scraped_data.xlsx')

In [57]:
print(f'Ads unscraped: {unscraped}')
print(len(urls_done))
print(len(unscraped_urls))

database.drop_duplicates(inplace=True)
print(database.shape)

print(len(urls))

Ads unscraped: 3
416
7
(490, 110)
498


In [58]:
database.sample(20)

,Address,Age of annuitant 1,Age of annuitant 2,Air conditioning,Alarm,As built plan,"As property tenant or owner, insure your real estate: fire, theft, water damage, storm, ...",Asking price excluding VAT and excl. notary fees (excluding eventual registration fees on the ground),Asking price excluding registration fees and excl. notary fees,Attic,...,Toilets,Town planning,Type of flood-risk area,Value of the property,Width of plot to street,Width of street façade,Wooded land,Workspace,Workspace area,Year of construction
0,Schommelaarstraat 11 | 9100 - NIEUWKERK...,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,2,NaN,NaN,NaN,13,9 m,NaN,NaN,NaN,2000
0,Jozef Wautersstraat 102 | 2830 - Willeb...,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,2,NaN,Property located in a non-flood zone,NaN,7,7 m,NaN,NaN,NaN,1956
0,Rue de Luxembourg 30 | 6720 - Habay,NaN,NaN,NaN,yes,Not specified,Choose your home insurance,NaN,,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1850
0,Chr. Plantijnstraat 9 | 3620 - Lanaken,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,2,NaN,Property located in a non-flood zone,NaN,NaN,NaN,NaN,NaN,NaN,1997
0,Roggestraat 41 | 2170 - Merksem,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,2,NaN,Recognized flood zone,NaN,6,6 m,NaN,NaN,NaN,1963
0,Laarderweg 14 | 3990 - Peer,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,1,NaN,Property located in a non-flood zone,NaN,NaN,NaN,NaN,NaN,NaN,1958
0,Olmenweg 21 | 3970 - Leopoldsburg,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,1,NaN,Property located in a non-flood zone,NaN,NaN,NaN,NaN,NaN,NaN,1930
0,Marvie 37 | 6600 - Bastogne,NaN,NaN,NaN,NaN,Not specified,Choose your home insurance,NaN,,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980
0,Route de Gosselies 294 | 6220 - Fleurus,NaN,NaN,NaN,NaN,no,Choose your home insurance,NaN,,NaN,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
0,Molenstraat 50 | 3500 - Hasselt,NaN,NaN,NaN,yes,no,Choose your home insurance,NaN,,NaN,...,2,NaN,Property located in a non-flood zone,NaN,NaN,NaN,NaN,NaN,NaN,1968
